# Model Training

## Requirements

In [1]:
## Load requirements
import pandas as pd
import plotly
import gc
import pickle
import matplotlib.pyplot as plt 

from typing import Tuple
from datetime import timedelta, datetime

from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
import pmdarima as pm

from utils import load, slicer

from sklearn.metrics import r2_score

## Data Load

In [2]:
df, resample_df = load()

## Model Training

### ARIMA (Auto ARIMA)

In [3]:
iter_max = 0
cont = 1
for stock in df["Name"].unique():
    iter_max += 1

## iterations over stock name in DF
for stock in df["Name"].unique():
    print("\nInitializing processess for: ", stock, "(",cont,"/",iter_max,")")
    #initialize model
    try: 
        model = pm.auto_arima(resample_df[(resample_df['Name']==stock)].drop(columns=["Name"])['Close'], 
                            m=12, seasonal=True,
                        start_p=0, start_q=0, max_order=4, test='adf',error_action='ignore',  
                            suppress_warnings=True,
                        stepwise=True, trace=True)
    except:
        print("Error in model initialization: ", stock)
        pass

    #Splitting
    try:
        df_train, df_val, df_test = slicer(resample_df[(resample_df['Name']==stock)])
    except:
        print("Error in splitting: ",stock)
        pass

    #model training & fitting
    try: 
        model.fit(df_train['Close'])
    except:
        print("Error in model fitting: ",stock)
        pass
    
    #export model to .pkl (pickle.dump)
    try: 
        file_name = (f'../stock_forecasting/AutoARIMA_Models/PKL/{stock}_autoARIMA_')
        # save
        pickle.dump(model, open(file_name + 'Model.pkl', "wb"))
    except:
        print("Error in model export: ",stock)
        pass

    gc.collect() #RAM usage optimization
    cont += 1


Initializing processess for:  MMM ( 1 / 31 )
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(1,0,1)[12] intercept   : AIC=10187.664, Time=2.86 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=10186.388, Time=0.08 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=10184.481, Time=0.64 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=10184.546, Time=1.08 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=10189.121, Time=0.07 sec
 ARIMA(1,1,0)(0,0,0)[12] intercept   : AIC=10182.507, Time=0.17 sec
 ARIMA(1,1,0)(0,0,1)[12] intercept   : AIC=10184.479, Time=0.76 sec
 ARIMA(1,1,0)(1,0,1)[12] intercept   : AIC=10183.787, Time=3.24 sec
 ARIMA(2,1,0)(0,0,0)[12] intercept   : AIC=10184.458, Time=0.21 sec
 ARIMA(1,1,1)(0,0,0)[12] intercept   : AIC=10184.485, Time=0.19 sec
 ARIMA(0,1,1)(0,0,0)[12] intercept   : AIC=10182.575, Time=0.13 sec
 ARIMA(2,1,1)(0,0,0)[12] intercept   : AIC=10186.367, Time=0.94 sec
 ARIMA(1,1,0)(0,0,0)[12]             : AIC=10185.671, Time=0.05 sec

Best model:  ARIMA(1,1,0)(